# Семинар 15
# Метод сопряжённых градиентов (Conjugate gradient method): гадкий утёнок 

## На прошлом семинаре

1. Метод Ньютона
2. Теоремы сходимости
4. Сравнение с градиентным спуском
5. Квазиньютоновские методы

## Система линейных уравнений vs. задача безусловной миимизации


## Как решать систему линейных уравнений $Ax = b$?

- Прямые методы основаны на матричных разложениях:
    - Плотная матрица $A$: для размерностей не больше нескольких тысяч
    - Разреженная (sparse) матрица $A$: для размерностей порядка $10^4 - 10^5$
- Итерационные методы: хороши для многих случаев, единственный подход для задач с размерностью $10^6$ и выше

## Немного истории...

M. Hestenes и E. Stiefel предложили *метод сопряжённых градиентов* для решения систем линейных уравнений в 1952 году как **прямой** метод. 

Также долгое время считалось, что метот представляет только теоретический интерес поскольку
- метод сопряжённых градиентов не работает на логарифмической линейке
- метод сопряжённых градиентов имеет небольшое преимущество перед исключением Гаусса при вычислениях на калькуляторе 

Подробнее [здесь](https://www.siam.org/meetings/la09/talks/oleary.pdf)

## Сопряжённые направления

## 

## Резюме

1. Сопряжённые направления
2. Метод сопряжённых градиентов [[1]](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf)
3. Сходимость
4. Сравнение с градиентным методом
5. Эксперименты